In [20]:
# Disable warnings, set Matplotlib inline plotting and load Pandas package
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import pandas as pd

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [21]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession


spark = SparkSession\
        .builder\
        .appName("Product_Categorisation")\
        .config('spark.sql.warehouse.dir','file:///home/sudipto21048867/Pyspark/spark-warehouse')\
        .getOrCreate()

In [22]:
TransactionDf = spark.read.csv('/user/sudipto21048867/Llyods_data/BASE_Data/TransactionSummaryLabelled.csv',                            
                          header='true', 
                          inferSchema='true', encoding='ISO-8859-1')

In [23]:
TransactionDf.show()

+---------+----+--------------------+-------+--------+---------+--------+--------------------+
|     Date|Type|         Description|Paid in|Paid out|  Balance|category|   Detailed Category|
+---------+----+--------------------+-------+--------+---------+--------+--------------------+
|25-Aug-17| POS|3818 23AUG17 , PA...|      -|   365.3|   £89.02|  Travel|            Agencies|
|25-Aug-17| POS|3818 23AUG17 , PA...|      -|   365.3|   £89.02|  Travel|            Agencies|
|22-Nov-17| POS|3818 21NOV17 , PA...|      -|   14.75|  £184.37|  Retail|            Clothing|
|18-Apr-17| POS|1177 13APR17 CD ,...|      -|    24.5|  £159.76|  Retail|            Clothing|
| 6-Feb-17| POS|3818 02FEB17 C , ...|      -|       2|  £737.19|  Retail|            Clothing|
| 7-Dec-16| POS|3818 05DEC16 , PR...|      -|      62|  £525.51|  Retail|            Clothing|
| 8-Feb-17| POS|3818 06FEB17 , PR...|      -|    32.3|    £9.41|  Retail|            Clothing|
|10-Jul-17| POS|3818 07JUL17 , PR...|      -|   11

In [24]:
TransactionDf.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Paid in: string (nullable = true)
 |-- Paid out: string (nullable = true)
 |-- Balance: string (nullable = true)
 |-- category: string (nullable = true)
 |-- Detailed Category: string (nullable = true)



In [25]:
from pyspark.sql.types import FloatType
from pyspark.sql.types import StringType
from pyspark.sql.functions import UserDefinedFunction as udf

In [26]:


paid_in_cleaned = udf(lambda y: y.replace(',', '') if y!='-' else '0' , StringType())
paid_out_cleaned = udf(lambda y: y.replace(',', '') if y!='-' else '0', StringType())
#paid_in_out_cleaned = udf(lambda k: paid_in_out_clean[k,l], FloatType(),FloatType())
#paid_paid_cleaned = udf(lambda k: paid_final_clean[k,l], FloatType())

In [27]:
TransactionDf = TransactionDf.withColumn('Paid in',paid_in_cleaned(TransactionDf['Paid in'])).\
                              withColumn('Paid out',paid_out_cleaned(TransactionDf['Paid out']))

In [28]:
TransactionDf.show()

+---------+----+--------------------+-------+--------+---------+--------+--------------------+
|     Date|Type|         Description|Paid in|Paid out|  Balance|category|   Detailed Category|
+---------+----+--------------------+-------+--------+---------+--------+--------------------+
|25-Aug-17| POS|3818 23AUG17 , PA...|      0|   365.3|   £89.02|  Travel|            Agencies|
|25-Aug-17| POS|3818 23AUG17 , PA...|      0|   365.3|   £89.02|  Travel|            Agencies|
|22-Nov-17| POS|3818 21NOV17 , PA...|      0|   14.75|  £184.37|  Retail|            Clothing|
|18-Apr-17| POS|1177 13APR17 CD ,...|      0|    24.5|  £159.76|  Retail|            Clothing|
| 6-Feb-17| POS|3818 02FEB17 C , ...|      0|       2|  £737.19|  Retail|            Clothing|
| 7-Dec-16| POS|3818 05DEC16 , PR...|      0|      62|  £525.51|  Retail|            Clothing|
| 8-Feb-17| POS|3818 06FEB17 , PR...|      0|    32.3|    £9.41|  Retail|            Clothing|
|10-Jul-17| POS|3818 07JUL17 , PR...|      0|   11

In [ ]:
TransactionDf['Paid in']=[str(x).replace(',', '') if x!='-' else '0' for x in TransactionDf['Paid in']]
TransactionDf['Paid out']=[str(x).replace(',', '') if x!='-' else '0' for x in TransactionDf['Paid out']]
TransactionDf[['Paid in','Paid out']]=TransactionDf[['Paid in','Paid out']].astype(float)
TransactionDf['final_paid']=(TransactionDf['Paid in']-TransactionDf['Paid out']).abs()